In [ ]:
import sys
import numpy as np
#from matplotlib.mlab import PCA
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn import svm
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

data = my_data = np.genfromtxt('full_pcm.results', delimiter=',')
#remove header and non-feature column
X = data[1:,4:]
Y = data[1:,3]

X = normalize(X, axis=0, norm='l2')


In [ ]:
# search for hyper parameters

gamma_list=[0.01,0.1,1,10,100,1000]
C_list=[0.01,0.1,1,10,100,1000]
w1_list=[]
w2_list=[]
#gamma_list=[1]
#C_list=[100]
# cross validation 
for g in gamma_list:
    for c in C_list:
        for w1 in w1_list:
            for w2 in w2_list:
                
                true_positive_count= 0
                false_positive_count= 0
                false_negative_count= 0
                for i in range(28):
                    train_X = np.concatenate((X[:i*8,:],X[i*8+8:,:]))
                    test_X = X[i*8:i*8+8,:]
                    train_Y = np.concatenate((Y[:i*8],Y[i*8+8:]))
                    test_Y = Y[i*8:i*8+8]
                    pca = PCA(n_components=5)
                    pca.fit(train_X)
                    #print(pca.components_)
                    #print(pca.explained_variance_ratio_)

                    transformed_train_X = pca.transform(train_X)
                    transformed_test_X = pca.transform(test_X)
                    #pca.components_

                    #learning

                    learner = svm.SVC(kernel='rbf', gamma=g, C=c, class_weight={0:w1, 1:w2})

                    learner.fit(transformed_train_X,train_Y)

                    predicted=learner.predict(transformed_test_X)
                    diff = predicted - test_Y
                    #print("results:"+ str(i))
                    #print(predicted)
                    #print(test_Y)

                    for index in range(len(predicted)):
                        if predicted[index] ==0 and test_Y[index] ==0:
                            true_positive_count +=1
                        if predicted[index] ==0 and test_Y[index] ==1:
                            false_positive_count +=1
                        if predicted[index] ==1 and test_Y[index] ==0:
                            false_negative_count += 1
                    print(diff)
                print("C = " + str(c) + ", gamma = " + str(g))
                print("recall = " + str(true_positive_count/(true_positive_count+false_negative_count)))
                print("precision = " + str(true_positive_count / (true_positive_count + false_positive_count)))
                print("\n")

